In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

# Import Dataset

In [ ]:
IMG_SIZE=
BATCH_SIZE=32

In [ ]:
def import_dataset(path, IMG_SIZE=256,BATCH_SIZE=32):
  dataset=tf.keras.preprocessing.image_dataset_from_directory(
      # path,
      shuffle=True,
      image_size=(IMG_SIZE,IMG_SIZE),
      batch_size=BATCH_SIZE
  )
  return dataset

In [ ]:
dataset=import_dataset(path,IMG_SIZE,BATCH_SIZE)

# Visualization

In [ ]:
class_names=dataset.class_names

In [ ]:
for image, label in dataset.table(1):
  print(image.shape)
  print(label.numpy())

In [ ]:
def visualize_data(dataset):
  plt.figure(figsize=(10,10))
  for image, label in dataset.table(1):
    for i in range(10):
      ax=plt.subplot(2,5,i+1)
      plt.imshow(image[i].numpy().astype("uint8"))
      plt.title(class_names[label[i]])
      plt.axis("off")

In [ ]:
visualize_data(dataset)

# Split the Data

In [ ]:
def train_test_split(dataset,train_split=0.8 , test_split=0.1, validation_split=0.1 ,shuffle=True,shuffle_size=10000):
  dataset_size=len(dataset)
  if shuffle:
    datatset=dataset.shuffle(shuffle_size,seed=10)

  train_size=int(dataset_size*train_split)
  val_size=int(validation_split*dataset_size)

  train_ds=dataset.take(train_size)
  val_ds=dataset.skip(train_size).take(val_size)
  test_ds=dataset.skip(train_size).skip(val_size)

  return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds=train_test_split(dataset)

- Check Class imbalance in train, valid and test

# Optimize the Data


In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

# Scale Data

In [ ]:
resize_and_scale=tf.keras.Sequential([
          layers.experimental.preprocessing.Resizing(IMG_SIZE,IMG_SIZE),
          layers.experimental.preprocessing.Rescaling(1/255)

])

# Data Augmentation

In [ ]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])